In [ ]:
import torch
from torch.nn import functional as F
from torch import nn
from pytorch_lightning.core.lightning import LightningModule

from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
import os
from torchvision import datasets, transforms
from torch.optim import Adam

from pytorch_lightning import Trainer

In [ ]:
class LitMNIST(LightningModule):

    def __init__(self):
        super().__init__()

        # mnist images are (1, 28, 28) (channels, width, height)
        self.layer_1 = torch.nn.Linear(28 * 28, 128)
        self.layer_2 = torch.nn.Linear(128, 256)
        self.layer_3 = torch.nn.Linear(256, 10)

    def forward(self, x):
        batch_size, channels, width, height = x.size()
        # (b, 1, 28, 28) -> (b, 1*28*28)
        x = x.view(batch_size, -1)
        
        # layer 1 (b, 1*28*28) -> (b, 128)
        x = self.layer_1(x)
        x = torch.relu(x)

        # layer 2 (b, 128) -> (b, 256)
        x = self.layer_2(x)
        x = torch.relu(x)

        # layer 3 (b, 256) -> (b, 10)
        x = self.layer_3(x)

        # probability distribution over labels
        x = torch.log_softmax(x, dim=1)

        return x
    
    def cross_entropy_loss(self, logits, labels):
        return F.nll_loss(logits, labels)

    def prepare_data(self):
        # transforms for images
        transform=transforms.Compose([transforms.ToTensor(), 
                                  transforms.Normalize((0.1307,), (0.3081,))])
        # prepare transforms standard to MNIST
        mnist_train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
        self.mnist_test = MNIST(os.getcwd(), train=False, download=True, transform=transform)
        
        self.mnist_train, self.mnist_val = random_split(mnist_train, [55000, 5000])
        
    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=64)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=64)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=64)

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=1e-3)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        
        logs = {'train_loss': loss}
        return {'loss': loss, 'log': logs}
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.cross_entropy_loss(logits, y)
        return {'val_loss': loss}
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.cross_entropy_loss(logits, y)
        
        return {'test_loss': loss}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        tensorboard_logs = {'test_loss': avg_loss}
        return {'test_loss': avg_loss, 'log': tensorboard_logs}

In [ ]:
net = LitMNIST()
x = torch.Tensor(1, 1, 28, 28)
out = net(x)

In [ ]:
out

In [ ]:
model = LitMNIST()
trainer = Trainer(max_epochs=3)
trainer.fit(model)

In [ ]:
trainer.test()

In [ ]:
model.logger

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

In [ ]:
dl = model.test_dataloader()

In [ ]:
dl

In [ ]:
for batch, lab in dl:
    print(batch.size())
    print(batch[0].numpy())
    break

In [ ]:
predictions = []
labels = []
for batch, lab in dl:
    pred = model(batch)
    pred = torch.argmax(pred, dim=1).numpy()
    predictions += list(pred)
    labels += list(lab.numpy())
    

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
cm = confusion_matrix(labels, predictions, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
cm

In [ ]:
accuracy_score(labels, predictions)

In [ ]:
dl = model.train_dataloader()
import matplotlib.pyplot as plt
for batch, lab in dl:
    print(lab.size())
    nimages = batch.size()[0]
    ncols = 4
    nrows = nimages // ncols
    print(ncols, nrows, nimages)
    f, axarr = plt.subplots(ncols, nrows)
    curr_col = 0
    for idx in range(nimages):
        col = idx % ncols
        row = idx // ncols
        img = batch[idx, :].numpy().squeeze()
        label = lab[idx]
        axarr[col, row].imshow(img)
    break
